In [ ]:
# !pip install comfit -q # Uncomment and run this cell if you are using Google Colab

If you find any errors in this tutorial, please raise a issue at [GitHub](https://github.com/vidarsko/ComFiT/issues).

# 2.1 - BoseEinsteinCondensate tutorial: comoving frame and defect tracking

Here we are going to show how the BoseEinsteinCondensate can be evolved in the comoving frame. We start by initialising a BoseEinsteinCondensate with a gaussian potential a bit forward in the computational domain. The ground state of this configuration is found by initialising the Thomas-Fermi ground state and evolving this in imaginary time.  

A constant (in time) potential is implemented in the code as the function bec.V_ext(t). The  default a function given as self.V_ext = lambda t: 0. This can be changed by using the function bec.conf_external_potential(V_ext, additive=False) 

We provide two functions that calculates some commonly used potentials. The gaussian and the harmonic potential. They are given by the functions:

bec.calc_Gaussian(position, width, top) and
bec.calc_harmonic_potential(R_tf)

where R_tf is the Thomas-Fermi radius (the size of the condensate, see documentation for details). 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import comfit as cf

bec = cf.BoseEinsteinCondensate(2,xRes=256,yRes=128,gamma=0,dt=0.05, plot_lib='matplotlib')

### task 1: Set the potiential to a constant gaussian at this position [bec.xmid+50,bec.ymid] with size = 5 and
# strength = 4
#Set the potential by defininf V0 = ...
# and calling the function bec.conf_external_potential(V0)

bec = cf.BoseEinsteinCondensate(2,xRes=256,yRes=128,gamma=0,dt=0.05, plot_lib='matplotlib')

pot = bec.calc_Gaussian(width = 5/np.sqrt(2),top=4, position = [bec.xmid+50,bec.ymid] )
bec.conf_external_potential(pot, additive=False)




We now need to initialise the wave function. A convinient starting point is the ground state of the system with the given potential. Since we don't have any analytic expression for this state we find it by starting out from a guess and remove energy by evolving in imaginary time (see previous notebook and documentation). The first guess is usualy the Thomas-Fermi ground state which is implemented as the function:

bec.conf_initial_condition_Thomas_Fermi()


In [ ]:
### task 2: Initialise the wavefunction using the Thomas-Fermi ground state and relax the system in imaginary time 
# for 50 time steps. Plot the absolut value squared of the wave function
#(Hint: the evolvers vere discussed in the previous notebook)

bec.conf_initial_condition_Thomas_Fermi()

bec.evolve_relax(50, method='ETD2RK') 


bec.plot_field(np.abs(bec.psi)**2,cmap_symmetric=False,colormap = 'winter')
plt.show()

Now we sett bec.gamma to be zero inside the computational domain and 1 on the edges. This can be done with the function bec.conf_dissipative_frame(d=7, frame_width_x=50, frame_width_y=50, frame_width_z=50). Here frame_width_x/frame_width_y/frame_width_z is the distance from the midle to the start of the dissipative frame in the x/y/z direction. In two dimensions frame_width_z is not used. d is the size of the interface between the bulk and the dissipative frame. 



In [ ]:
#### task 3. make a dissipative frame with d=7, frame_width_x = 100 and frame_width_y = 50. plot bec.gamma

bec.conf_dissipative_frame(frame_width_x=100,frame_width_y=50)

bec.plot_field(bec.gamma,cmap_symmetric=False,colormap = 'winter')
plt.show()



Now we want to evolve the system in the comoving frame and we want to use the function bec.calc_vortex_nodes() to find the vortices.

In [ ]:
### task 4. evolve the system in the comoving frame with vel_x = 0.4. Make an animation of the absolut value squared 
# of the wavefunction and mark the position of the defects.   



vel_x = 0.40



N= 800

for n in range(N):
    psi_prev = np.copy(bec.psi)
    
    bec.evolve_comoving_dGPE(10,vel_x,method='ETD4RK')
    
    dt_psi = (bec.psi - psi_prev)/(bec.dt*10)
    nodes = bec.calc_vortex_nodes(dt_psi=dt_psi)

    fig,ax=bec.plot_field(np.abs(bec.psi)**2,cmap_symmetric=False,colormap = 'gray')
    
    bec.plot_nodes(nodes,ax=ax)
    bec.plot_save(n,fig)

cf.tool_make_animation_gif(n)


In [ ]:
### We can now plot the vortices you traced 

fig, ax=bec.plot_field(np.abs(bec.psi)**2,cmap_symmetric=False,colormap = 'gray')
bec.plot_nodes(nodes,ax=ax)
plt.show()